In [123]:
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import numpy as np

In [223]:
# 读取训练数据
origin_train_df = pd.read_csv("[new] yancheng_train_20171226.csv")
# 读取预测数据
origin_test_df = pd.read_csv("yancheng_testA_20171225.csv")

In [224]:
origin_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20157 entries, 0 to 20156
Data columns (total 32 columns):
sale_date                20157 non-null int64
class_id                 20157 non-null int64
sale_quantity            20157 non-null int64
brand_id                 20157 non-null int64
compartment              20157 non-null int64
type_id                  20157 non-null int64
level_id                 20157 non-null object
department_id            20157 non-null int64
TR                       20157 non-null object
gearbox_type             20157 non-null object
displacement             20157 non-null float64
if_charging              20157 non-null object
price_level              20157 non-null object
price                    20157 non-null object
driven_type_id           20157 non-null int64
fuel_type_id             20157 non-null object
newenergy_type_id        20157 non-null int64
emission_standards_id    20157 non-null int64
if_MPV_id                20157 non-null int64
if_luxur

In [225]:
# 预处理
def charge_price_level(level):
    if "-" in level:
        # 价格区间
        min_price = int(level.split("-")[0])
        max_price = int(level.split("-")[1][:-1])
        return (min_price, max_price, (min_price + max_price) / 2.0)
    else:
        # 低于……价格
        return (0, int(level[:-2]), float(level[:-2]) / 2)
    

def charge_power(power):
    power = str(power)
    if power == "-":
        return -1
    if "/" in power:
        # 分数表示法（两种类型）
        return (float(power.split("/")[0]) + float(power.split("/")[1])) / 2
    else:
        return float(power)

def charge_rated_passenger(value):
    value = str(value)
    if "-" in value:
        return (float(value.split("-")[1]) + float(value.split("-")[0])) / 2
    else:
        return int(value)

origin_train_df["level_id"] = origin_train_df["level_id"].apply(lambda x: 0 if x == '-' else int(x))
origin_train_df["department_id"] = origin_train_df["department_id"].apply(lambda x: str(x))
origin_train_df["min_price"] = origin_train_df["price_level"].apply(lambda x: charge_price_level(x)[0])
origin_train_df["max_price"] = origin_train_df["price_level"].apply(lambda x: charge_price_level(x)[1])
origin_train_df["avg_price"] = origin_train_df["price_level"].apply(lambda x: charge_price_level(x)[2])
origin_train_df["driven_type_id"] = origin_train_df["driven_type_id"].apply(lambda x: str(x))
origin_train_df["fuel_type_id"] = origin_train_df["fuel_type_id"].apply(lambda x: str(x))
origin_train_df["newenergy_type_id"] = origin_train_df["newenergy_type_id"].apply(lambda x: str(x))
origin_train_df["emission_standards_id"] = origin_train_df["emission_standards_id"].apply(lambda x: str(x))
origin_train_df["power"] = origin_train_df["power"].apply(charge_power)
origin_train_df["engine_torque"] = origin_train_df["engine_torque"].apply(charge_power)
origin_train_df["rated_passenger"] = origin_train_df["rated_passenger"].apply(charge_rated_passenger)

In [226]:
import copy
# 标签列
ori_label_name = "sale_quantity"
# 需要聚合后计算均值的列
mean_lst_names = ["sale_date", "class_id", "compartment", "displacement", "avg_price", "power", 
                  "cylinder_number", "engine_torque", "car_length", "car_width", "car_height", "total_quality", 
                  "equipment_quality", "rated_passenger", "wheelbase", "front_track", "rear_track", "max_price",
                  "min_price"]

# 需要聚合后计算种类数的字段
count_lst_names = ["sale_date", "class_id", "compartment", "type_id", "level_id", "department_id", "TR", 
                   "gearbox_type", "displacement", "if_charging", "driven_type_id", "fuel_type_id", 
                   "newenergy_type_id", "emission_standards_id", "if_MPV_id", "if_luxurious_id", "rated_passenger",
                   "max_price", "min_price", "avg_price"]

# 加权平均的列
weight_mean_lst_name = ["compartment", "displacement", "power", "cylinder_number", "engine_torque", "car_length", "car_width",
                "car_height", "total_quality", "equipment_quality", "rated_passenger", "wheelbase", "front_track", 
                "rear_track", "max_price", "min_price", "avg_price"]
label_df = origin_train_df[["sale_date", "class_id", ori_label_name]].groupby(["sale_date", "class_id"]).sum()
# 取字段均值
mean_df = origin_train_df[mean_lst_names].groupby(["sale_date", "class_id"]).aggregate(np.mean)
# 取字段类型计数，例如在sale_date，class_id下有多少种箱型
count_df = origin_train_df[count_lst_names].groupby(["sale_date", "class_id"]).aggregate(
    lambda lst: len(set(lst.values)))
# 取字段和label的加权平均
weight_mean_lst_name_tmp = [name + "_label" for name in weight_mean_lst_name]
origin_train_tmp = origin_train_df.copy()
for name in weight_mean_lst_name:
    origin_train_tmp[name + "_label"] = origin_train_tmp[name] * origin_train_tmp[ori_label_name]
weight_mean_names = copy.copy(weight_mean_lst_name_tmp)
weight_mean_names.append(ori_label_name)
weight_mean_names.append("sale_date")
weight_mean_names.append("class_id")
weight_mean_df = origin_train_tmp[weight_mean_names].groupby(["sale_date", "class_id"]).sum().copy()
for name in weight_mean_lst_name_tmp:
    weight_mean_df[name] = weight_mean_df[name] / weight_mean_df[ori_label_name]

In [227]:
train_df = label_df.copy()
mean_train_names = [name + "_mean" for name in mean_lst_names[2:]]
count_train_names = [name + "_count" for name in count_lst_names[2:]]
weight_mean_train_names = [name + "_mean" for name in weight_mean_lst_name_tmp]
train_df[mean_train_names] = mean_df[mean_lst_names[2:]]
train_df[count_train_names] = count_df[count_lst_names[2:]]
train_df[weight_mean_train_names] = weight_mean_df[weight_mean_lst_name_tmp]
train_df["sales"] = label_df[ori_label_name]
train_df

sale_quantity  compartment_mean  displacement_mean  \
sale_date class_id                                                       
201201    125403               90          2.000000           2.600000   
          136916              665          3.000000           2.271429   
          178529             1739          1.000000           1.087500   
          194450               65          2.000000           2.000000   
          198427               47          3.000000           1.500000   
          209945              168          3.000000           2.750000   
          248352             1713          3.000000           1.466667   
          281301              965          3.000000           1.680000   
          290854              632          2.000000           2.200000   
          291086              507          3.000000           2.133333   
          304458              499          3.000000           1.700000   
          321683              478          3.000000           1.600000   
          349023             2522          3.000000           1.650000   
          354068              460          3.000000           1.733333   
          379876              245          3.000000           1.800000   
          384924              738          3.000000           2.350000   
          395234              102          3.000000           2.100000   
          409854              120          2.000000           2.000000   
          425432             1994          3.000000           1.600000   
          453752               96          3.000000           2.250000   
          468392              969          3.000000           2.133333   
          472603              319          3.000000           1.733333   
          482497              182          2.000000           2.000000   
          527809              199          3.000000           2.200000   
          559132              428          3.000000           2.250000   
          576298              301          2.000000           1.900000   
          580470              467          3.000000           1.500000   
          580634              506          2.000000           1.600000   
          597927             1321          3.000000           2.133333   
          603972              513          3.000000           2.528571   
...                           ...               ...                ...   
201710    745137              852          2.666667           1.600000   
          750340               95          2.000000           1.500000   
          760412               66          2.000000           2.000000   
          786351              104          3.000000           1.600000   
          789290               77          3.000000           1.500000   
          810398              114          2.000000           1.600000   
          815230              168          2.000000           1.400000   
          819061               94          2.000000           2.000000   
          842246              121          2.000000           1.800000   
          851857              157          2.000000           2.000000   
          854079              162          3.000000           1.800000   
          854548              122          2.000000           1.500000   
          861459              252          3.000000           1.500000   
          871642              155          2.000000           1.500000   
          872180              127          2.000000           2.000000   
          883691              254          3.000000           1.800000   
          890189             1302          2.000000           1.500000   
          905061              363          2.000000           1.550000   
          905745              183          2.000000           1.400000   
          914348              483          2.000000           1.466667   
          923841              642          3.000000           1.500000   
          924154        

In [37]:
# 建模方式如下：
# 特征是历史的销售报表，标签是这个月的销售量。就像作为销售经理，用当月报表预测下个月汽车销量一样。
# “历史”这个词可以有多种跨度：上个月，过去半年，过去一年，过去三年，历史全部。需要注意有些车型没有上市这么长时间（最短只上了2个月）
# 暂时只考虑过去一个月的销售情况，构造特征的时候可以增加上个月销量和其他信息的交叉特征
# 注意验证集构造，可以用原始训练集最后一个月的销售情况作验证集，其中有4个上市两个月的车型被排除在这个验证集外，可以最后再单独考虑这四个车型。

In [38]:
!mkdir baseline

In [228]:
train_df = train_df.swaplevel(0, 1, axis=0)

In [229]:
train_df["index"] = train_df.index
train_df["sale_date"] = train_df["index"].apply(lambda item: item[1])
train_df["class_id"] = train_df["index"].apply(lambda item: item[0])
train_df["sale_date"] = train_df["sale_date"].apply(lambda date: (date / 100 - 2012) * 12 + date % 100)
del train_df["index"]

In [230]:
train_tmp = train_df.copy()
train_tmp["label"] = -1
label_df = train_tmp[["sale_quantity", "sale_date", "class_id"]]
for index in train_df.index:
    date = (index[1] / 100 - 2012) * 12 + index[1] % 100
    class_id = index[0]
#     train_tmp.loc[index, "label"] = label_df[(label_df["sale_date"] == date - 1) & 
#                                              (label_df["class_id"] == class_id)]["sale_quantity"].values[0]
    label = label_df[(label_df["sale_date"] == date + 1) & (label_df["class_id"] == class_id)]["sale_quantity"].values
    if len(label) > 0:
        train_tmp.loc[index, "label"] = label[0]
train_tmp.head()

,,sale_quantity,compartment_mean,displacement_mean,avg_price_mean,power_mean,cylinder_number_mean,engine_torque_mean,car_length_mean,car_width_mean,car_height_mean,...,wheelbase_label_mean,front_track_label_mean,rear_track_label_mean,max_price_label_mean,min_price_label_mean,avg_price_label_mean,sales,sale_date,class_id,label
class_id,sale_date,,,,,,,,,,,,,,,,,,,,,
125403,201201,90,2.0,2.600000,42.5,145.333333,4.666667,246.666667,5241.666667,1867.666667,1764.666667,...,3088.000000,1593.000000,1601.0,50.0,35.0,42.5,90,1,125403,156
136916,201201,665,3.0,2.271429,17.5,121.857143,4.000000,214.142857,4825.000000,1822.857143,1482.142857,...,2775.000000,1575.000000,1560.0,20.0,15.0,17.5,665,1,136916,351
178529,201201,1739,1.0,1.087500,2.5,54.250000,4.000000,96.500000,3940.625000,1568.125000,1860.000000,...,2505.750431,1282.426682,1290.0,5.0,0.0,2.5,1739,1,178529,1493
194450,201201,65,2.0,2.000000,17.5,104.000000,4.000000,184.000000,4325.000000,1795.000000,1680.000000,...,2630.000000,1550.000000,1550.0,20.0,15.0,17.5,65,1,194450,59
198427,201201,47,3.0,1.500000,6.5,79.000000,4.000000,141.000000,4602.000000,1725.000000,1485.000000,...,2602.000000,1482.000000,1462.0,8.0,5.0,6.5,47,1,198427,21


In [232]:
train_tmp.to_csv("./baseline/train_origin.csv", index=None)

In [233]:
train_origin = pd.read_csv("./baseline/train_origin.csv")
train_X = train_origin[train_origin["label"] >= 0].copy()
print train_X.columns
train_y = train_X["label"]
del train_X["sale_quantity"]
del train_X["class_id"]
del train_X["label"]
del train_X["sale_date"]
test_X = train_origin[train_origin["sale_date"] == 70].copy()
class_id = test_X["class_id"]
del test_X["sale_quantity"]
del test_X["class_id"]
del test_X["label"]
del test_X["sale_date"]
cv_train_X = train_origin[(train_origin["label"] >= 0) & (train_origin["sale_date"] < 69)].copy()
cv_train_y = cv_train_X["label"]
del cv_train_X["sale_quantity"]
del cv_train_X["class_id"]
del cv_train_X["label"]
del cv_train_X["sale_date"]
cv_test_X = train_origin[(train_origin["label"] >= 0) & (train_origin["sale_date"] == 69)].copy()
cv_test_y = cv_test_X["label"]
del cv_test_X["sale_quantity"]
del cv_test_X["class_id"]
del cv_test_X["label"]
del cv_test_X["sale_date"]

Index([u'sale_quantity', u'compartment_mean', u'displacement_mean',
       u'avg_price_mean', u'power_mean', u'cylinder_number_mean',
       u'engine_torque_mean', u'car_length_mean', u'car_width_mean',
       u'car_height_mean', u'total_quality_mean', u'equipment_quality_mean',
       u'rated_passenger_mean', u'wheelbase_mean', u'front_track_mean',
       u'rear_track_mean', u'max_price_mean', u'min_price_mean',
       u'compartment_count', u'type_id_count', u'level_id_count',
       u'department_id_count', u'TR_count', u'gearbox_type_count',
       u'displacement_count', u'if_charging_count', u'driven_type_id_count',
       u'fuel_type_id_count', u'newenergy_type_id_count',
       u'emission_standards_id_count', u'if_MPV_id_count',
       u'if_luxurious_id_count', u'rated_passenger_count', u'max_price_count',
       u'min_price_count', u'avg_price_count', u'compartment_label_mean',
       u'displacement_label_mean', u'power_label_mean',
       u'cylinder_number_label_mean', u'engine_

In [234]:
# 用训练集训练的模型
dtrain = xgb.DMatrix(train_X.values, train_y)
dtest = xgb.DMatrix(test_X.values)
watchlist = [(dtrain, "train")]
xgb_param = {'max_depth':4, 'eta':0.02, 'silent':1, 'objective':'reg:linear', "subsample": 0.8, "num_round": 3000}
bst = xgb.train(xgb_param, dtrain, xgb_param['num_round'], watchlist, verbose_eval=10)
test_y = bst.predict(dtest)
result_df = train_origin[train_origin["sale_date"] == 70][["class_id"]].copy()
result_df["predict_quantity"] = test_y
result_df["predict_date"] = "201711"
result_df["predict_quantity"] = result_df["predict_quantity"].apply(lambda x: int(x))
result_df[["predict_date", "class_id", "predict_quantity"]].to_csv("./baseline/baseline_result4.csv", index=None)

[0]	train-rmse:711.884
[10]	train-rmse:600.43
[20]	train-rmse:511.698
[30]	train-rmse:441.698
[40]	train-rmse:386.676
[50]	train-rmse:344.613
[60]	train-rmse:311.871
[70]	train-rmse:286.973
[80]	train-rmse:268.7
[90]	train-rmse:255.424
[100]	train-rmse:245.011
[110]	train-rmse:237.365
[120]	train-rmse:231.623
[130]	train-rmse:226.82
[140]	train-rmse:223.065
[150]	train-rmse:219.95
[160]	train-rmse:217.152
[170]	train-rmse:214.704
[180]	train-rmse:211.911
[190]	train-rmse:209.585
[200]	train-rmse:207.95
[210]	train-rmse:206.037
[220]	train-rmse:204.088
[230]	train-rmse:202.196
[240]	train-rmse:200.362
[250]	train-rmse:198.891
[260]	train-rmse:197.127
[270]	train-rmse:195.816
[280]	train-rmse:194.414
[290]	train-rmse:193.059
[300]	train-rmse:191.848
[310]	train-rmse:190.823
[320]	train-rmse:189.849
[330]	train-rmse:188.687
[340]	train-rmse:187.651
[350]	train-rmse:186.658
[360]	train-rmse:185.754
[370]	train-rmse:184.766
[380]	train-rmse:184
[390]	train-rmse:182.928
[400]	train-rmse:182.

In [235]:
# 用验证集训练模型
dtrain_cv = xgb.DMatrix(cv_train_X.values, cv_train_y)
dtest_cv = xgb.DMatrix(cv_test_X.values, cv_test_y)
watchlist_cv = [(dtest_cv, "test_cv")]
xgb_param = {'max_depth':4, 'eta':0.02, 'silent':1, 'objective':'reg:linear', "subsample": 0.8, "num_round": 3000}
model_cv = xgb.train(xgb_param, dtrain_cv, xgb_param['num_round'], watchlist_cv, verbose_eval=10)
model_cv.predict(dtest_cv)

[0]	test_cv-rmse:565.748
[10]	test_cv-rmse:461.616
[20]	test_cv-rmse:380.097
[30]	test_cv-rmse:315.69
[40]	test_cv-rmse:264.902
[50]	test_cv-rmse:225.668
[60]	test_cv-rmse:196.696
[70]	test_cv-rmse:178.468
[80]	test_cv-rmse:164.631
[90]	test_cv-rmse:157.472
[100]	test_cv-rmse:153.082
[110]	test_cv-rmse:149.091
[120]	test_cv-rmse:148.361
[130]	test_cv-rmse:148.541
[140]	test_cv-rmse:148.354
[150]	test_cv-rmse:148.706
[160]	test_cv-rmse:149.059
[170]	test_cv-rmse:150.429
[180]	test_cv-rmse:150.942
[190]	test_cv-rmse:152.033
[200]	test_cv-rmse:152.493
[210]	test_cv-rmse:152.83
[220]	test_cv-rmse:154.34
[230]	test_cv-rmse:154.407
[240]	test_cv-rmse:155.2
[250]	test_cv-rmse:156.422
[260]	test_cv-rmse:155.951
[270]	test_cv-rmse:156.437
[280]	test_cv-rmse:157.506
[290]	test_cv-rmse:157.619
[300]	test_cv-rmse:156.816
[310]	test_cv-rmse:157.153
[320]	test_cv-rmse:157.38
[330]	test_cv-rmse:157.115
[340]	test_cv-rmse:156.997
[350]	test_cv-rmse:157.219
[360]	test_cv-rmse:158.144
[370]	test_cv-rmse

[2990]	test_cv-rmse:162.919
[2999]	test_cv-rmse:162.988


array([  366.07800293,   308.70553589,   304.50585938,   203.76771545,
         260.56713867,   263.81939697,   194.82565308,   116.7858963 ,
         224.96632385,   277.18496704,   133.83164978,  1711.29370117,
         302.98800659,   170.57170105,   304.82254028,   139.88621521,
         281.60424805,   180.03077698,   283.06414795,   425.14614868,
         842.12609863,   603.1619873 ,   332.93984985,   290.04846191,
         258.89813232,   449.19366455,   407.71707153,   108.17043304,
         147.3301239 ,   486.6723938 ,   156.40811157,  2186.69506836,
         337.26937866,  2283.20141602,   433.97280884,  1077.80163574,
         514.69622803,   730.62347412,   557.13421631,    79.28079224,
         159.60621643,   185.23735046,   457.31304932,   138.62886047,
         329.90670776,   217.97364807,   660.53094482,   245.25361633,
         137.53663635,   185.93652344,  1741.64160156,   171.54310608,
         271.92526245,   120.7644043 ,  1957.98205566,   463.21313477,
      

In [ ]:
# 特征从几个角度出发：脑洞，badcase，特征深挖，注意结合特征筛选尽可能通过验证集尝试
